In [1]:
import numpy as np
import pandas as pd
import json
import requests
from pandas.io.json import json_normalize
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge beautifulsoup4 --yes
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
print("Libraries imported.")

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - beautifulsoup4


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    beautifulsoup4-4.9.1       |   py36h9f0ad1d_0         163 KB  conda-forge
    openssl-1.1.1g             |       h516909a_0         2.1 MB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.6 MB

The following NEW packages will be INSTALLED:

    python_abi:      3.6-1_cp36m       conda-forge

The following packages will be UPDATED:

    beautifulsoup4:  4.7.1-py36_1                

In [2]:
!conda install -c conda-forge folium=0.5.0 --yes
import folium
from pandas.io.json import json_normalize
data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(data,'html.parser')
postalCodeList = []
boroughList = []
neighborhoodList = []

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - folium=0.5.0


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    branca-0.4.1               |             py_0          26 KB  conda-forge
    folium-0.5.0               |             py_0          45 KB  conda-forge
    vincent-0.4.4              |             py_1          28 KB  conda-forge
    altair-4.1.0               |             py_1         614 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         713 KB

The following NEW packages will be INSTALLED:

    altair:  4.1.0-py_1 conda-forge
    branca:  0.4.1-py_0 conda-forge
    folium:  0.5.0-py_0 conda-forge
    vincent: 0.4.4-py_1 conda-forge


branca-0.4.1         | 26 KB     | ##################################### | 1

In [3]:
soup.find('table').find_all('tr')
for row in soup.find('table').find_all('tr'):
    cells = row.find_all('td')
    if(len(cells) > 0):
        postalCodeList.append(cells[0].text)
        boroughList.append(cells[1].text)
        neighborhoodList.append(cells[2].text.rstrip('\n'))
toronto_df = pd.DataFrame({"Postalcode": postalCodeList,
                            "Borough": boroughList,
                            "Neighborhood": neighborhoodList})
toronto_df.head()

,Postalcode,Borough,Neighborhood
0,M1A\n,Not assigned\n,Not assigned
1,M2A\n,Not assigned\n,Not assigned
2,M3A\n,North York\n,Parkwoods
3,M4A\n,North York\n,Victoria Village
4,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"


In [4]:
toronto_df.replace("Not assigned\n",np.nan, inplace=True)
toronto_df = toronto_df[toronto_df["Borough"].notnull()]
toronto_df.reset_index(drop=True, inplace=True)
toronto_df.head()

,Postalcode,Borough,Neighborhood
0,M3A\n,North York\n,Parkwoods
1,M4A\n,North York\n,Victoria Village
2,M5A\n,Downtown Toronto\n,"Regent Park, Harbourfront"
3,M6A\n,North York\n,"Lawrence Manor, Lawrence Heights"
4,M7A\n,Downtown Toronto\n,"Queen's Park, Ontario Provincial Government"


In [5]:
toronto_df_grouped = toronto_df.groupby(["Postalcode","Borough"], as_index=False).agg(lambda x: ",".join(x))
toronto_df_grouped.head()

,Postalcode,Borough,Neighborhood
0,M1B\n,Scarborough\n,"Malvern, Rouge"
1,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
2,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae


In [6]:
toronto_df_grouped = toronto_df.groupby(["Postalcode","Borough"], as_index=False).agg(lambda x: ", ".join(x))
toronto_df_grouped.head()

,Postalcode,Borough,Neighborhood
0,M1B\n,Scarborough\n,"Malvern, Rouge"
1,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
2,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae


In [7]:
for index, row in toronto_df_grouped.iterrows():
    if row["Neighborhood"] == "Not assigned":
        row["Neighborhood"] = row ["Borough"]

toronto_df_grouped.head()

,Postalcode,Borough,Neighborhood
0,M1B\n,Scarborough\n,"Malvern, Rouge"
1,M1C\n,Scarborough\n,"Rouge Hill, Port Union, Highland Creek"
2,M1E\n,Scarborough\n,"Guildwood, Morningside, West Hill"
3,M1G\n,Scarborough\n,Woburn
4,M1H\n,Scarborough\n,Cedarbrae


# QUESTION 1 :Scrape the Wikipedia page to obtain the data that is in the table of postal codes and to transform the data into a pandas dataframe 

In [8]:
Column_names = ["Postalcode","Borough", "Neighborhood"]
test_df = pd.DataFrame(columns=Column_names)
test_list = ["M5G\n","M2H\n","M4B\n","M1J\n","M4G\n","M4M\n", "M1R\n","M9V\n", "M9L\n", "M5V\n", "M1B\n", "M5A\n"]
for postcode in test_list :
    test_df = test_df.append(toronto_df_grouped[toronto_df_grouped["Postalcode"]==postcode], ignore_index=True)
test_df

,Postalcode,Borough,Neighborhood
0,M5G\n,Downtown Toronto\n,Central Bay Street
1,M2H\n,North York\n,Hillcrest Village
2,M4B\n,East York\n,"Parkview Hill, Woodbine Gardens"
3,M1J\n,Scarborough\n,Scarborough Village
4,M4G\n,East York\n,Leaside
5,M4M\n,East Toronto\n,Studio District
6,M1R\n,Scarborough\n,"Wexford, Maryvale"
7,M9V\n,Etobicoke\n,"South Steeles, Silverstone, Humbergate, Jamest..."
8,M9L\n,North York\n,Humber Summit
9,M5V\n,Downtown Toronto\n,"CN Tower, King and Spadina, Railway Lands, Har..."


# Question 2 : Get the latitude and the longitude coordinates of each neighborhood.

In [9]:
# The code was removed by Watson Studio for sharing.

,Postalcode,Latitude,Longitude
0,M1B\n,43.806686,-79.194353
1,M1C\n,43.784535,-79.160497
2,M1E\n,43.763573,-79.188711
3,M1G\n,43.770992,-79.216917
4,M1H\n,43.773136,-79.239476


In [10]:
df_row_merged = pd.concat([test_df, df_data_1], axis=1)
df_data_2 = df_row_merged.dropna(axis=0, how="any", thresh=None, subset=None,inplace=False)
df_data_2

,Postalcode,Borough,Neighborhood,Postalcode,Latitude,Longitude
0,M5G\n,Downtown Toronto\n,Central Bay Street,M1B\n,43.806686,-79.194353
1,M2H\n,North York\n,Hillcrest Village,M1C\n,43.784535,-79.160497
2,M4B\n,East York\n,"Parkview Hill, Woodbine Gardens",M1E\n,43.763573,-79.188711
3,M1J\n,Scarborough\n,Scarborough Village,M1G\n,43.770992,-79.216917
4,M4G\n,East York\n,Leaside,M1H\n,43.773136,-79.239476
5,M4M\n,East Toronto\n,Studio District,M1J\n,43.744734,-79.239476
6,M1R\n,Scarborough\n,"Wexford, Maryvale",M1K\n,43.727929,-79.262029
7,M9V\n,Etobicoke\n,"South Steeles, Silverstone, Humbergate, Jamest...",M1L\n,43.711112,-79.284577
8,M9L\n,North York\n,Humber Summit,M1M\n,43.716316,-79.239476
9,M5V\n,Downtown Toronto\n,"CN Tower, King and Spadina, Railway Lands, Har...",M1N\n,43.692657,-79.264848


# Question 3 : Explore and cluster the neighborhoods in Toronto and generate maps to visualize your neighborhoods and how they cluster together.

In [2]:
# To imprint the search for latitude and longitude for Borough co-ordinates onto map using Foursquare API

In [14]:
CLIENT_ID = "DB5UMIQ0UFAALZQ2AG2AOMB1V5MG05UKJ02YFXSSWWZ2ZD2B"
CLIENT_SECRET = "NMBUSEVZQAEYVE3RZGAOBUOWEPG4SEAVP2RON0PJ4S2G0IQS"
VERSION = '20180604'
LIMIT = 30
print("Your credentials:")
print("CLIENT_ID: "+ CLIENT_ID)
print("CLIENT_SECRET: "+ CLIENT_SECRET)

Your credentials:
CLIENT_ID: DB5UMIQ0UFAALZQ2AG2AOMB1V5MG05UKJ02YFXSSWWZ2ZD2B
CLIENT_SECRET: NMBUSEVZQAEYVE3RZGAOBUOWEPG4SEAVP2RON0PJ4S2G0IQS


In [17]:
address = "Borough, Toronto"

geolocator = Nominatim(user_agent = "foursquare_agent")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

51.42936555 0.10139469856643134


In [25]:
search_query = "Toronto"
radius = 500
print(search_query + ' ..... OK!')
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ii={},{}&v={}&query={}&radius={}&limit{}'.format(CLIENT_ID,CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

Toronto ..... OK!


'https://api.foursquare.com/v2/venues/search?client_id=DB5UMIQ0UFAALZQ2AG2AOMB1V5MG05UKJ02YFXSSWWZ2ZD2B&client_secret=NMBUSEVZQAEYVE3RZGAOBUOWEPG4SEAVP2RON0PJ4S2G0IQS&ii=51.42936555,0.10139469856643134&v=20180604&query=Toronto&radius=500&limit30'

In [44]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=13)
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color = 'red',
    popup = 'Conrad',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)
venues_map